In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import pymysql
import time
import re


NUMBER_OF_POSTS =  # 1000
LOG_FILE = "C:\\Users\\ <REMOVED> \\joblog.txt"
SPACER = "\n\n\n ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ "
SPACER2 = "\n===================================================="
PASSWORD = "<REMOVED>" 
MATCHDATA_FILE = "C:\\Users\\ <REMOVED> \\matchData.txt"

try:
    conn = pymysql.connect(host="<REMOVED>", port=3306, user="<REMOVED>", passwd=PASSWORD, db="<REMOVED>")
    cur = conn.cursor()
except:
    pass


pageLink1 = "https://www.google.com/search?vet=<REMOVED>&client=firefox-b-1-d&yv=3&rciv=jb&nfpr=0&q="
pageLink2 = "&start="
pageLink3 = "&asearch=jb_list&async=_id:VoQFxe,_pms:hts,_fmt:pc"

titles = ["software+engineer", "data+analyst", "web+developer", "database+developer"]


def search(TITLE):
    global output
    output = ""  # compiles all results from multiple soups bc we are using XHRs to infinitely scroll
    counter = 0
    while counter < NUMBER_OF_POSTS:
        counter += 10
        URL = pageLink1 + TITLE + pageLink2 + str(counter) + pageLink3
        print("URL: ", URL)
        hdr = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
        req = Request(URL, headers=hdr)
        html = urlopen(req)
        soup = BeautifulSoup(html, "html.parser")
        output = output+str(soup)
        time.sleep(.025)


def getLinks():
    global dataSet, matchData
    txtloc = re.finditer("(?<=line-height:1.5em\">).+?(?=<\/span><\/)", str(output), re.DOTALL)
    link = re.finditer('(?<=href=")([^\'" >]+)(?=" jsaction="trigger)', str(output))
    matchData = []  # list with text start position and description
    urlData = []  # list with link start position and url
    dataSet = []  # list of lists, each of which hold links to index-corresponding description
    for words in txtloc:
        matchData.append([words.start(), words[0]])
    for i in link:
        urlData.append([i.start(), i[0]])
    for text in matchData:
        block = []
        dataSet.append(block)
        for url in urlData:
            if url[0] <= text[0] and url[1] != "#":
                block.append(url[1])
                url[0] = 999999999999999


def getText():
    global descrpt, clean_out
    clean_out = []
    descrpt = re.findall('(?<=line-height:1.5em">).+?(?=<\/span><\/)', str(output), flags=re.DOTALL)
    for i in range(len(descrpt)):
        if re.search('(</span></div>)', str(descrpt[i]), flags=re.DOTALL):
            clean_out.append(re.findall('.*(?=</span></div><div)', descrpt[i], flags=re.DOTALL))
        else:
            clean_out.append(re.findall('.*', str(descrpt[i]), flags=re.DOTALL))
    for i in range(len(clean_out)):
        clean_out[i] = re.sub(r'(<span)(.*)(\">)', ' ', str(clean_out[i][0]))


oldSet = []
indexCounter = 1
sqlIDcounter = 1


def sqlURL(TITLE, indexCounter):
    global sqlIDcounter
    for eachURL in dataSet:
        if eachURL[0] not in oldSet:
            cur.execute("insert into jobposting (url,category) values ( %s,%s)", (str(eachURL[0]), str(TITLE)))
            # print(cur.fetchall())
            conn.commit()
            oldSet.append(eachURL[0])
            sqlSKILLS(indexCounter)
            sqlIDcounter += 1
            indexCounter += 1


def sqlSKILLS(indexCounter):
    try:
        pattern = re.compile(r'python{0,5}', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 1))
    except:
        pass
    try:
        pattern = re.compile(r'sql{0,5}', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 2))
    except:
        pass
    try:
        pattern = re.compile(r'\bexcel\b', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 3))
    except:
        pass
    try:
        pattern = re.compile(r'C\+\+', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 4))
    except:
        pass
    try:
        pattern = re.compile(r'\bjava\b', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 5))
    except:
        pass
    try:
        pattern = re.compile(r'\bgithub\b', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 6))
    except:
        pass
    try:
        pattern = re.compile(r'\php\b', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 7))
    except:
        pass
    try:
        pattern = re.compile(r'\bjavascript\b', flags=re.IGNORECASE)
        matches = pattern.findall(matchData[indexCounter-1][1])
        for match in matches:
            cur.execute("insert into kw_intersect (jobPosting_id, keywords_id) values (%s,%s)", (sqlIDcounter, 8))
    except:
        pass


def logOutput():
    with open(LOG_FILE, 'a') as f:
        for i in range(len(descrpt)):
            try:
                print(SPACER, file=f)
                print(TITLE, file=f)
                print("JOB NUMBER: ", i+1, file=f)
                print(clean_out[i], file=f)
                print(SPACER2, file=f)
                print(dataSet[i], file=f)
            except:
                pass
    f.close()


def logMatchData(TITLE):
    with open(MATCHDATA_FILE, "a") as f:
        for item in matchData:
            try:
                f.write('%s\n' % item)
            except:
                pass


if __name__ == "__main__":
    for TITLE in titles:
        search(TITLE)
        getLinks()
        getText()
        logOutput()
        logMatchData(TITLE)
        sqlURL(TITLE, indexCounter)
        print("TOTAL NUMBER OF ITEMS: ", len(clean_out))

cur.close()
conn.close()
